##### Vectorization Pipeline Components:
1. Chroma Integration
2. Semantic Document Preparation
3. Semantic Search Testing
4. Database Analytics

##### Usage Flow:
- Input: classification/results/output.csv from classification/loyalty_classifier_nb notebook
- Processing: Convert semantic classifications to embeddings
- Output: Searchable vector database for RAG queries

##### Test Queries Included:
- "loyalty points calculation rules"
- "order processing workflow"
- "customer data integration"
- "payment service integration"
- "business rule patterns"

In [1]:
CLASSIFICATION_CSV = "../classification/results/{llm}/output.csv"
DB_PATH = "results/{embedder}/chroma_db"
COLLECTION_NAME = "loyalty_code_semantics_{llm}"
# "all-MiniLM-L6-v2",      # Fast, good general purpose
# "all-mpnet-base-v2",     # Better quality, slower
# "multi-qa-MiniLM-L6-cos-v1"  # Optimized for Q&A

def setup(llm: str, embedder: str):
    config = { "llm": llm, "embedder": embedder }

    classification_csv = CLASSIFICATION_CSV.format(**config)
    db_path = DB_PATH.format(**config)
    collection_name = COLLECTION_NAME.format(**config)

    return classification_csv, db_path, collection_name

In [2]:
from vectorization.document_utils import prepare_documents_for_embedding, load_classification_data
import json

from vectorization.semantic_vector_database import SemanticVectorDatabase

def run_vectorization_pipeline(db_path: str,
                                    classification_csv: str,
                                    collection_name: str,
                                    embedding_model: str,
                                    reset_db: bool = True):
    """Main pipeline to create vector database from classification results"""

    print("=== CodeSense Vector Database Creation ===")

    # Initialize database
    vector_db = SemanticVectorDatabase(db_path, embedding_model)

    # Create collection
    collection = vector_db.create_collection(collection_name, reset_db)

    # Prepare documents from classification data
    df = load_classification_data(classification_csv)
    documents = prepare_documents_for_embedding(df)

    # Add to collection
    collection.add_documents_to_collection(documents)

    # Get collection statistics
    stats = collection.get_collection_stats_v1()
    print(f"\n=== Collection Statistics ===")
    print(json.dumps(stats, indent=2, default=str))

    # Test semantic search
    test_queries = [
        "loyalty points calculation rules",
        "order processing workflow",
        "customer data integration",
        "payment service integration",
        "business rule patterns"
    ]

    print(f"\n=== Testing Semantic Search ===")
    for query in test_queries:
        collection.semantic_search(query, n_results=3)

In [ ]:
print("\n=== Running Vectorization with Anthropic set and  all-MiniLM-L6-v2 ===")
classification_csv, db_path, collection_name = setup("claude3.5", "all-MiniLM-L6-v2")
run_vectorization_pipeline(db_path=db_path, classification_csv=classification_csv, collection_name=collection_name, embedding_model="all-MiniLM-L6-v2")

In [ ]:
print("\n=== Running Vectorization with Anthropic set and  all-mpnet-base-v2 ===")
classification_csv, db_path, collection_name = setup("claude3.5", "all-mpnet-base-v2")
run_vectorization_pipeline(db_path=db_path, classification_csv=classification_csv, collection_name=collection_name, embedding_model="all-mpnet-base-v2")

In [ ]:
print("\n=== Running Vectorization with Ollama set and  all-MiniLM-L6-v2 ===")
classification_csv, db_path, collection_name = setup("codellama", "all-MiniLM-L6-v2")
run_vectorization_pipeline(db_path=db_path, classification_csv=classification_csv, collection_name=collection_name, embedding_model="all-MiniLM-L6-v2")

In [ ]:
print("\n=== Running Vectorization with Ollama set and  all-mpnet-base-v2 ===")
classification_csv, db_path, collection_name = setup("codellama", "all-mpnet-base-v2")
run_vectorization_pipeline(db_path=db_path, classification_csv=classification_csv, collection_name=collection_name, embedding_model="all-mpnet-base-v2")

In [ ]:
print("\n=== Running Vectorization with OpenAI set and  all-MiniLM-L6-v2 ===")
classification_csv, db_path, collection_name = setup("gpt4.1", "all-MiniLM-L6-v2")
run_vectorization_pipeline(db_path=db_path, classification_csv=classification_csv, collection_name=collection_name, embedding_model="all-MiniLM-L6-v2")

In [ ]:
print("\n=== Running Vectorization with OpenAI set and  all-mpnet-base-v2 ===")
classification_csv, db_path, collection_name = setup("gpt4.1", "all-mpnet-base-v2")
run_vectorization_pipeline(db_path=db_path, classification_csv=classification_csv, collection_name=collection_name, embedding_model="all-mpnet-base-v2")